In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/VKR_Malceva')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data

In [2]:
print("="*60)
print("ЗАГРУЗКА И ПОДГОТОВКА ДАННЫХ ДЛЯ LIGHTGBM")
print("="*60)

# 1. Загрузка данных
print("1. Загрузка данных...")
train_df = pd.read_csv('./data/train_processed.csv')
val_df = pd.read_csv('./data/val_processed.csv')

# 2. Разделение на признаки и таргет
print("2. Разделение данных...")
X_train = train_df.drop(columns=['TARGET'])
y_train = train_df['TARGET']
X_val = val_df.drop(columns=['TARGET'])
y_val = val_df['TARGET']

# 3. Исправление названий столбцов для LightGBM
print("3. Исправление названий столбцов...")

def fix_column_names(df):
    """Исправляет названия столбцов для совместимости с LightGBM"""
    df_fixed = df.copy()
    new_columns = {}
    
    for col in df_fixed.columns:
        # Заменяем все не-буквенно-цифровые символы на подчеркивание
        new_name = str(col)
        # Заменяем пробелы и основные проблемные символы
        for char in [' ', '(', ')', '[', ']', '{', '}', '/', '\\', ':', ',', '.', '-', '+', '*', '&', '^', '%', '$', '#', '@', '!', '?']:
            new_name = new_name.replace(char, '_')
        
        # Убираем множественные подчеркивания
        while '__' in new_name:
            new_name = new_name.replace('__', '_')
        
        # Убираем подчеркивания в начале/конце
        new_name = new_name.strip('_')
        
        # Если имя пустое или начинается с цифры
        if not new_name:
            new_name = f'feature_{col}'
        elif new_name[0].isdigit():
            new_name = f'f_{new_name}'
        
        new_columns[col] = new_name
    
    return df_fixed.rename(columns=new_columns)

X_train = fix_column_names(X_train)
X_val = fix_column_names(X_val)

print(f"   Переименовано {X_train.shape[1]} столбцов")
print(f"   Пример: '{list(train_df.columns)[0]}' -> '{X_train.columns[0]}'")

# 4. Оптимизация типов данных
print("4. Оптимизация типов данных...")

# Целевая переменная -> int8
y_train = y_train.astype(np.int8)
y_val = y_val.astype(np.int8)

# Бинарные признаки (0/1) -> int8
binary_mask = (X_train.nunique() <= 2) & (X_train.isin([0, 1]).all())
binary_cols = X_train.columns[binary_mask].tolist()

if binary_cols:
    X_train[binary_cols] = X_train[binary_cols].astype(np.int8)
    X_val[binary_cols] = X_val[binary_cols].astype(np.int8)
    print(f"   Бинарных признаков: {len(binary_cols)} -> int8")

# Остальные признаки -> float32
other_cols = [col for col in X_train.columns if col not in binary_cols]
if other_cols:
    X_train[other_cols] = X_train[other_cols].astype(np.float32)
    X_val[other_cols] = X_val[other_cols].astype(np.float32)
    print(f"   Непрерывных признаков: {len(other_cols)} -> float32")

# 5. Удаление признаков с нулевой дисперсией
print("5. Удаление признаков с нулевой дисперсией...")
zero_var_cols = X_train.columns[X_train.std() == 0]
if len(zero_var_cols) > 0:
    X_train = X_train.drop(columns=zero_var_cols)
    X_val = X_val.drop(columns=zero_var_cols)
    print(f"   Удалено {len(zero_var_cols)} признаков с нулевой дисперсией")
else:
    print("   Признаков с нулевой дисперсией не найдено")

# 6. Очистка памяти
print("6. Очистка памяти...")
del train_df, val_df
import gc
gc.collect()

# 7. Итоговые результаты
print("\n" + "="*60)
print("ИТОГОВЫЕ РАЗМЕРЫ ДАННЫХ")
print("="*60)

print(f"X_train: {X_train.shape} (память: {X_train.memory_usage().sum()/1024**2:.1f} MB)")
print(f"X_val:   {X_val.shape} (память: {X_val.memory_usage().sum()/1024**2:.1f} MB)")
print(f"y_train: {y_train.shape} (тип: {y_train.dtype})")
print(f"y_val:   {y_val.shape} (тип: {y_val.dtype})")

print(f"\nТипы признаков в X_train:")
for dtype in X_train.dtypes.unique():
    count = (X_train.dtypes == dtype).sum()
    print(f"  {dtype}: {count} признаков")

print(f"\nПроверка пропущенных значений:")
print(f"  X_train: {X_train.isna().sum().sum()}")
print(f"  X_val:   {X_val.isna().sum().sum()}")

print(f"\nПримеры названий признаков:")
print(f"  {list(X_train.columns[:3])}")

print("\n" + "="*60)
print("ДАННЫЕ ГОТОВЫ ДЛЯ LIGHTGBM!")
print("="*60)

ЗАГРУЗКА И ПОДГОТОВКА ДАННЫХ ДЛЯ LIGHTGBM
1. Загрузка данных...
2. Разделение данных...
3. Исправление названий столбцов...
   Переименовано 702 столбцов
   Пример: 'SK_ID_CURR' -> 'SK_ID_CURR'
4. Оптимизация типов данных...
   Бинарных признаков: 115 -> int8
   Непрерывных признаков: 587 -> float32
5. Удаление признаков с нулевой дисперсией...
   Удалено 5 признаков с нулевой дисперсией
6. Очистка памяти...

ИТОГОВЫЕ РАЗМЕРЫ ДАННЫХ
X_train: (184506, 697) (память: 432.5 MB)
X_val:   (61502, 697) (память: 144.2 MB)
y_train: (184506,) (тип: int8)
y_val:   (61502,) (тип: int8)

Типы признаков в X_train:
  float32: 587 признаков
  int8: 110 признаков

Проверка пропущенных значений:
  X_train: 0
  X_val:   0

Примеры названий признаков:
  ['SK_ID_CURR', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL']

ДАННЫЕ ГОТОВЫ ДЛЯ LIGHTGBM!


# CatBoost

In [5]:
# В начале ноутбука добавь эту ячейку
print("="*60)
print("УСТАНОВКА CATBOOST ДЛЯ COLAB")
print("="*60)

try:
    import catboost
    print(f"✅ CatBoost уже установлен (версия: {catboost.__version__})")
except ImportError:
    print("❌ CatBoost не найден. Устанавливаю...")
    !pip install catboost -q
    print("✅ CatBoost успешно установлен!")
    import catboost
    print(f"   Версия: {catboost.__version__}")

УСТАНОВКА CATBOOST ДЛЯ COLAB
❌ CatBoost не найден. Устанавливаю...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 7.7 MB/s eta 0:00:00:00:0100:01
✅ CatBoost успешно установлен!
   Версия: 1.2.8


In [7]:
print("="*80)
print("БЛОК 1: ПОИСК ЛУЧШИХ ГИПЕРПАРАМЕТРОВ CATBOOST (ИСПРАВЛЕННАЯ ВЕРСИЯ)")
print("="*80)

import numpy as np
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.model_selection import RandomizedSearchCV
import warnings
warnings.filterwarnings('ignore')

# 1. Создаем базовую модель с фиксированными безопасными параметрами
print("\n1. Создание базовой модели CatBoost...")
base_cat = CatBoostClassifier(
    random_seed=42,
    verbose=False,
    thread_count=-1,
    allow_writing_files=False,
    loss_function='Logloss',
    eval_metric='AUC',
    # Фиксируем параметры, которые часто вызывают проблемы
    bootstrap_type='Bernoulli',  # Самый стабильный тип
    subsample=0.8,                # Фиксированный subsample
    grow_policy='SymmetricTree',  # Самый стабильный
    score_function='L2'           # L2 - самая стабильная
)

# 2. УПРОЩЕННАЯ И БЕЗОПАСНАЯ сетка гиперпараметров
print("\n2. Определение безопасной сетки гиперпараметров...")
param_distributions = {
    # Только основные параметры
    'iterations': [300, 400, 500, 600],
    'learning_rate': [0.03, 0.05, 0.1, 0.15],
    'depth': [4, 5, 6, 7, 8],
    'l2_leaf_reg': [1, 3, 5, 7, 9],
    
    # Убираем проблемные параметры
    # 'max_leaves' - убираем
    # 'grow_policy' - фиксируем
    # 'bootstrap_type' - фиксируем
    # 'score_function' - фиксируем
    # 'bagging_temperature' - убираем
    # 'bayesian_matrix_reg' - убираем
    # 'rsm' - убираем
    # 'min_data_in_leaf' - убираем
    
    # Только эти параметры для баланса классов
    'auto_class_weights': [None, 'Balanced'],
}

print(f"   Всего параметров для поиска: {sum(len(v) if isinstance(v, list) else 1 for v in param_distributions.values())}")
print(f"   Параметры: {list(param_distributions.keys())}")

# 3. Настройка RandomizedSearchCV
print("\n3. Настройка RandomizedSearchCV...")
random_search = RandomizedSearchCV(
    estimator=base_cat,
    param_distributions=param_distributions,
    n_iter=15,           # Уменьшаем до 15 итераций для теста
    cv=3,
    scoring='roc_auc',
    n_jobs=1,
    verbose=10,
    random_state=42,
    refit=False,
    return_train_score=True
)

# 4. Запуск поиска
print("\n4. Запуск поиска...")
print(f"   Будет проверено {15 * 3} моделей")
print("-"*80)

import time
start_time = time.time()
random_search.fit(X_train, y_train)
search_time = time.time() - start_time

# 5. Результаты
print("\n" + "="*80)
print("РЕЗУЛЬТАТЫ ПОИСКА")
print("="*80)

print(f"Поиск завершен за {search_time/60:.1f} минут")
print(f"\nЛучшие параметры:")
for param, value in random_search.best_params_.items():
    print(f"  {param}: {value}")
print(f"\nЛучший ROC-AUC на кросс-валидации: {random_search.best_score_:.4f}")

# 6. Сохраняем параметры
best_params = random_search.best_params_

import json
import joblib

with open('./models/catboost_best_params.json', 'w') as f:
    json.dump(best_params, f, indent=2)

print(f"\n✅ Параметры сохранены в ./models/catboost_best_params.json")

БЛОК 1: ПОИСК ЛУЧШИХ ГИПЕРПАРАМЕТРОВ CATBOOST (ИСПРАВЛЕННАЯ ВЕРСИЯ)

1. Создание базовой модели CatBoost...

2. Определение безопасной сетки гиперпараметров...
   Всего параметров для поиска: 20
   Параметры: ['iterations', 'learning_rate', 'depth', 'l2_leaf_reg', 'auto_class_weights']

3. Настройка RandomizedSearchCV...

4. Запуск поиска...
   Будет проверено 45 моделей
--------------------------------------------------------------------------------
Fitting 3 folds for each of 15 candidates, totalling 45 fits
[CV 1/3; 1/15] START auto_class_weights=Balanced, depth=7, iterations=500, l2_leaf_reg=9, learning_rate=0.03
[CV 1/3; 1/15] END auto_class_weights=Balanced, depth=7, iterations=500, l2_leaf_reg=9, learning_rate=0.03;, score=(train=0.846, test=0.776) total time= 3.4min
[CV 2/3; 1/15] START auto_class_weights=Balanced, depth=7, iterations=500, l2_leaf_reg=9, learning_rate=0.03
[CV 2/3; 1/15] END auto_class_weights=Balanced, depth=7, iterations=500, l2_leaf_reg=9, learning_rate=0.03

In [3]:
print("="*80)
print("БЛОК 2: ОБУЧЕНИЕ ФИНАЛЬНОЙ МОДЕЛИ CATBOOST")
print("="*80)

import json
import joblib
import numpy as np
import pandas as pd
from catboost import CatBoostClassifier
import time

# 1. Загрузка лучших параметров
print("\n1. Загрузка лучших параметров...")
try:
    with open('./models/catboost_best_params.json', 'r') as f:
        best_params = json.load(f)
    print(f"✅ Загружено {len(best_params)} параметров")
    print(f"Параметры: {best_params}")
except FileNotFoundError:
    print("❌ Файл с параметрами не найден!")
    print("Использую параметры по умолчанию...")
    best_params = {
        'iterations': 500,
        'learning_rate': 0.1,
        'depth': 6,
        'l2_leaf_reg': 3
    }

# 2. Создание финальной модели
print("\n2. Создание финальной модели CatBoost...")
final_cat = CatBoostClassifier(
    **best_params,
    random_seed=42,
    thread_count=-1,
    verbose=50,           # Вывод каждые 50 итераций
    allow_writing_files=False,
    loss_function='Logloss',
    eval_metric='AUC'
)

# 3. Обучение на всех тренировочных данных
print(f"\n3. Обучение на всех {len(X_train)} samples...")
print(f"   Размер данных: {X_train.shape}")
print(f"   Количество признаков: {X_train.shape[1]}")

train_start = time.time()
final_cat.fit(X_train, y_train)
train_time = time.time() - train_start

print(f"\n✅ Модель обучена за {train_time/60:.1f} минут")
print(f"   Количество итераций: {final_cat.tree_count_}")

# 4. Сохраняем модель
print("\n4. Сохранение модели...")
final_cat.save_model('./models/catboost_final_model.cbm')
joblib.dump(final_cat, './models/catboost_final_model.pkl')
print("✅ Модель сохранена в двух форматах:")
print(f"   - CatBoost native: ./models/catboost_final_model.cbm")
print(f"   - Pickle: ./models/catboost_final_model.pkl")

БЛОК 2: ОБУЧЕНИЕ ФИНАЛЬНОЙ МОДЕЛИ CATBOOST

1. Загрузка лучших параметров...
✅ Загружено 5 параметров
Параметры: {'learning_rate': 0.05, 'l2_leaf_reg': 7, 'iterations': 500, 'depth': 5, 'auto_class_weights': 'Balanced'}

2. Создание финальной модели CatBoost...

3. Обучение на всех 184506 samples...
   Размер данных: (184506, 697)
   Количество признаков: 697
0:	total: 236ms	remaining: 1m 57s
50:	total: 2.77s	remaining: 24.4s
100:	total: 5.18s	remaining: 20.5s
150:	total: 7.67s	remaining: 17.7s
200:	total: 10s	remaining: 14.9s
250:	total: 12.4s	remaining: 12.3s
300:	total: 14.8s	remaining: 9.75s
350:	total: 17.2s	remaining: 7.3s
400:	total: 19.7s	remaining: 4.86s
450:	total: 22.2s	remaining: 2.41s
499:	total: 24.6s	remaining: 0us

✅ Модель обучена за 0.4 минут
   Количество итераций: 500

4. Сохранение модели...
✅ Модель сохранена в двух форматах:
   - CatBoost native: ./models/catboost_final_model.cbm
   - Pickle: ./models/catboost_final_model.pkl


In [4]:
print("="*80)
print("БЛОК 3: ОЦЕНКА МЕТРИК CATBOOST")
print("="*80)

import joblib
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, roc_auc_score, precision_score, recall_score

# 1. Загрузка модели
print("\n1. Загрузка обученной модели...")
final_cat = joblib.load('./models/catboost_final_model.pkl')
print("✅ Модель загружена")

# 2. Предсказания на валидационной выборке
print(f"\n2. Предсказания на валидации ({len(X_val)} samples)...")
y_val_pred = final_cat.predict(X_val)
y_val_pred_proba = final_cat.predict_proba(X_val)[:, 1]

# 3. Расчет метрик
print("\n3. Расчет метрик...")
accuracy = accuracy_score(y_val, y_val_pred)
precision = precision_score(y_val, y_val_pred, average='macro', zero_division=0)
recall = recall_score(y_val, y_val_pred, average='macro', zero_division=0)
f1 = f1_score(y_val, y_val_pred, average='macro', zero_division=0)
roc_auc = roc_auc_score(y_val, y_val_pred_proba)

# 4. Вывод результатов
print("\n" + "="*60)
print("РЕЗУЛЬТАТЫ CATBOOST НА ВАЛИДАЦИОННОЙ ВЫБОРКЕ")
print("="*60)

print(f"\nОсновные метрики:")
print(f"  Accuracy:   {accuracy:.4f}")
print(f"  Precision:  {precision:.4f}")
print(f"  Recall:     {recall:.4f}")
print(f"  F1 Macro:   {f1:.4f}")
print(f"  ROC-AUC:    {roc_auc:.4f}")

# 5. Classification Report
print("\n" + "-"*60)
print("Classification Report:")
print("-"*60)
print(classification_report(y_val, y_val_pred, zero_division=0))

# 6. Confusion Matrix
print("\n" + "-"*60)
print("Confusion Matrix:")
print("-"*60)
cm = confusion_matrix(y_val, y_val_pred)
print(cm)

# 7. Сохранение финальной модели с метриками
print("\n" + "="*60)
print("СОХРАНЕНИЕ ФИНАЛЬНОЙ МОДЕЛИ С МЕТРИКАМИ")
print("="*60)

# Загружаем лучшие параметры
try:
    with open('./models/catboost_best_params.json', 'r') as f:
        best_params = json.load(f)
except:
    best_params = {}

# Подготовка данных для сохранения
catboost_final_artifacts = {
    'model': final_cat,
    'best_params': best_params,
    'val_metrics': {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_macro': f1,
        'roc_auc': roc_auc,
        'confusion_matrix': cm.tolist()
    },
    'training_info': {
        'model_type': 'CatBoost',
        'training_time_minutes': train_time / 60 if 'train_time' in locals() else None,
        'iterations': final_cat.tree_count_,
        'feature_count': X_train.shape[1],
        'training_samples': len(X_train),
        'validation_samples': len(X_val),
        'timestamp': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')
    }
}

# Сохраняем только модель и метрики
joblib.dump(catboost_final_artifacts, './models/catboost_final_model_with_metrics.pkl')
print("✅ Финальная модель с метриками сохранена: './models/catboost_final_model_with_metrics.pkl'")

# 8. Итоговая информация
print("\n" + "="*80)
print("ИТОГОВАЯ ИНФОРМАЦИЯ О CATBOOST МОДЕЛИ")
print("="*80)

print(f"\nМодель: CatBoost Classifier")
print(f"Валидационный ROC-AUC: {roc_auc:.4f}")
print(f"Accuracy на валидации: {accuracy:.4f}")
print(f"F1 Macro на валидации: {f1:.4f}")

print(f"\nХарактеристики модели:")
print(f"  Количество итераций: {final_cat.tree_count_}")
print(f"  Глубина деревьев: {best_params.get('depth', 'N/A')}")
print(f"  Learning rate: {best_params.get('learning_rate', 'N/A')}")

print(f"\n" + "="*80)
print("CATBOOST МОДЕЛЬ ОБУЧЕНА И ОЦЕНЕНА!")
print("="*80)

БЛОК 3: ОЦЕНКА МЕТРИК CATBOOST

1. Загрузка обученной модели...
✅ Модель загружена

2. Предсказания на валидации (61502 samples)...

3. Расчет метрик...

РЕЗУЛЬТАТЫ CATBOOST НА ВАЛИДАЦИОННОЙ ВЫБОРКЕ

Основные метрики:
  Accuracy:   0.7171
  Precision:  0.5692
  Recall:     0.7002
  F1 Macro:   0.5518
  ROC-AUC:    0.7714

------------------------------------------------------------
Classification Report:
------------------------------------------------------------
              precision    recall  f1-score   support

           0       0.96      0.72      0.82     56537
           1       0.18      0.68      0.28      4965

    accuracy                           0.72     61502
   macro avg       0.57      0.70      0.55     61502
weighted avg       0.90      0.72      0.78     61502


------------------------------------------------------------
Confusion Matrix:
------------------------------------------------------------
[[40727 15810]
 [ 1589  3376]]

СОХРАНЕНИЕ ФИНАЛЬНОЙ МОДЕЛИ С М

In [5]:
# 1. В начале ноутбука создаем пустой список
model_results = []

# 2. После КАЖДОЙ обученной модели:
def log_model(model, X_val, y_val, name):
    """Записывает метрики модели в таблицу"""
    
    y_pred = model.predict(X_val)
    
    result = {
        'model': name,
        'accuracy': round(accuracy_score(y_val, y_pred), 4),
        'f1': round(f1_score(y_val, y_pred, average='macro'), 4)
    }
    
    # ROC-AUC если доступен
    if hasattr(model, 'predict_proba'):
        try:
            y_proba = model.predict_proba(X_val)
            if len(model.classes_) == 2:
                result['roc_auc'] = round(roc_auc_score(y_val, y_proba[:, 1]), 4)
        except:
            pass
    
    model_results.append(result)
    
    # Создаем DataFrame и показываем
    results_df = pd.concat([pd.read_csv('results.csv'), pd.DataFrame(model_results)], ignore_index=True)
    print(f"\nДобавлена модель: {name}")
    print(results_df.to_string(index=False))
    
    return results_df

In [6]:

artifacts = joblib.load('./models/catboost_final_model_with_metrics.pkl')

results = log_model(artifacts['model'], X_val, y_val, 'CatBoost')
results.to_csv('results.csv', index=False)


Добавлена модель: CatBoost
                 model  accuracy     f1  roc_auc
         SGDClassifier    0.7016 0.5416   0.7617
 DcisionTreeClassifier    0.6481 0.5034   0.7082
RandomForestClassifier    0.9171 0.5336   0.7524
              LightGBM    0.7645 0.5776   0.7733
               XGBoost    0.8653 0.6237   0.7726
              CatBoost    0.7171 0.5518   0.7714
